In [1]:
from mesa import Agent, Model
from mesa.visualization.ModularVisualization import ModularServer
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
from mesa.space import MultiGrid
from mesa.space import Grid
from mesa.visualization.UserParam import UserSettableParameter

from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.modules import ChartModule, TextElement
from mesa.datacollection import DataCollector
import numpy as np
import math
import random

In [2]:

def find_bikes_req(num_boxes,ratio_v_b,v_parcels,b_parcels):
    b_req =math.ceil(((num_boxes)/(b_parcels+(v_parcels*ratio_v_b)))) #ratio van to bike should be between 0-5
    return b_req

def find_bikes_dec(num_boxes,ratio_v_b,v_parcels,b_parcels):
    b_req =(((num_boxes)/(b_parcels+(v_parcels*ratio_v_b))))
    return b_req

def find_vans_req(num_boxes,ratio_v_b,v_parcels,b_parcels):
    b_req = find_bikes_dec(num_boxes,ratio_v_b,v_parcels,b_parcels)
    v_req= math.ceil(b_req*ratio_v_b)
    return v_req


Functions that the car type agents will share

In [3]:
def move(self):
    #function for movement along road, only cars are using this            
        x, y = self.pos
                
        """"if self.model.grid.is_cell_empty([x-1, y])== True:
            self.model.grid.move_agent(self, (x-1, y))
            print('neighbors: %s' %(neighbors,)) """
        #check which road agent is on now
        #top_road, left_road, sec_road, third_road, bot_road
        if self.pos in top_road:
            #print( 'my position is: %s'%(self.pos,))
            if self.model.grid.is_cell_empty([x-1, y])== True: #always check if cell ahead is empty or not
                self.model.grid.move_agent(self, (x-1, y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
        
                
        elif self.pos in left_road:
            if self.model.grid.is_cell_empty([x, y-1])== True:
                self.model.grid.move_agent(self, (x, y-1))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif (self.pos in sec_road) or (self.pos in third_road) or (self.pos in bot_road):
            if self.model.grid.is_cell_empty([x+1, y])== True:
                self.model.grid.move_agent(self, (x+1, y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos in right_road:
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (x, y+1))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
                
        #turning points
        elif self.pos == (2,29):
            if self.model.grid.is_cell_empty([x, y-1])== True:
                self.model.grid.move_agent(self, (2,28))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif (self.pos == (2,11)) or (self.pos == (2,20)): 
            if self.model.grid.is_cell_empty([x, y-1]) and self.model.grid.is_cell_empty([x+1, y]) == True:
                possible_steps = ((x, y-1),(x+1, y))
                move_to = self.random.choice(possible_steps)
                self.model.grid.move_agent(self, move_to)
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (2,0): 
            if self.model.grid.is_cell_empty([x+1, y])== True:
                self.model.grid.move_agent(self, (3,0))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,11): 
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (27,12))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,20): 
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (27,21))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,29): 
            if self.model.grid.is_cell_empty([x-1, y])== True:
                self.model.grid.move_agent(self, (x-1,y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,0): 
                if len(self.parcels) != 0: 
                    #later add delivery status
                    self.model.grid.move_agent(self, (27,1))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (28,0))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                    
        elif self.pos == (28,0): 
            self.model.grid.move_agent(self, (29,0))
            self.spm=1
            self.tot_spm_made+=1
            self.tot_spm_possible+=1
            
        else:
            print('I am stuck on the road, my pos is %s' %(self.pos,))
            
#parking, bikes can park on any curb
def park(self):
        x, y = self.pos   
        if (self.pos in top_road) or (self.pos in sec_road) or (self.pos in third_road): #look diagonally
            if self.state==1: #1 is moving, 0 is stationary
                t = self.model.grid.get_cell_list_contents([(x, y-1)])
                if len(t) <= 1:
                    self.model.grid.move_agent(self, (x, y-1))#move downward 1 step to park
                    self.state=0
                
        elif self.pos in left_road:
            if self.state==1: #1 is moving, 0 is stationary
                t=self.model.grid.get_cell_list_contents([(x-1, y)])
                if len(t) <= 1:
                    self.model.grid.move_agent(self, (x-1, y))#move leftward 1 step
                    self.state=0
        
        elif self.pos in right_road:
            if self.state==1: #1 is moving, 0 is stationary
                t=self.model.grid.get_cell_list_contents([(x+1, y)])
                if len(t) <= 1:
                    self.model.grid.move_agent(self, (x+1, y))#move rightward 1 step
                    self.state=0
                
#function to depart after delivery                    
def depart(self):
        x, y =self.pos
        #for pos in top, second, third road
        if (self.pos in [(i,28) for i in range(3,27)] ) or (self.pos in [(i,19) for i in range(3,27)] ) or (self.pos in [(i,10) for i in range(3,27)] ):
            self.model.grid.move_agent(self, (x, y+1))
            self.state=1
                
        elif self.pos in [(1,i) for i in range(0,30)]: #left road
            self.model.grid.move_agent(self, (x+1, y))#move downward 1 step to park
            self.state=1
        
        elif self.pos in [(28,i) for i in range(1,29)]: #right road
            self.model.grid.move_agent(self, (x-1, y))#move downward 1 step to park
            self.state=1
            
def get_curb(self,pos): #returns a list of curb agents
        for i in self.model.grid.get_cell_list_contents([pos]):
            if type(i) is curb:
                return i

# Road

In [4]:
class road(Agent):
    
        
    def __init__(self, pos, model):
        super().__init__(pos, model)
        
        global top_road 
        global left_road
        global sec_road
        global third_road
        global bot_road
        global right_road
        global exit_path
        top_road = [(i, 29) for i in range(3,30)]    
        left_road = [(2, i) for i in range(1,11)] +[(2, i) for i in range(12,20)] + [(2, i) for i in range(21,29)]
        sec_road = [(i, 20) for i in range(3,27)] 
        third_road = [(i, 11) for i in range(3,27)] 
        bot_road = [(i, 0) for i in range(3,27)] #+ [(i, 0) for i in range(28,29)]
        right_road = [(27, i) for i in range(1,11)] +[(27, i) for i in range(12,20)] + [(27, i) for i in range(21,29)]
        exit_path = [(i, 0) for i in range(28,29)]
        
       

# Curb

In [5]:
class curb(Agent):
    def __init__(self, pos, model):
        super().__init__(pos, model) # 1-100%
        self.dist_calc={} #this is probably useless
        self.delivered_by_b=[] #parcel list delivered by bikes
        self.delivered_by_v=[]#by vans
        
        #2 empty lists for building agent n bike agent to append
        self.van_del=[] #this is for vans to check, but currently left blank becuz i hvent set up loading_bay for vans
        self.bike_del=[] #juz a name, just by both bikes and vans to check arrival
        
        #vans can only park in the 2nd and 3rd lane
        mid_curb = [(i, 19) for i in range(3,27)] 
        bot_curb = [(i, 10) for i in range(3,27)]
        
        #list to contain the times used for the whole delivery process for all agents
        self.dtime = []
        self.dtime_v = []
        
                 
    def step(self):
        #if len(self.bike_del)!=0:
        #    print("Curbs: To receive parcels# %s" %(self.bike_del), " , received %s" %(self.delivered_by_b))
        #if len(self.dtime)!=0:
        #    print("Curbs: dtime %s" %(self.dtime))
        return

# Building

In [6]:
class target_building(Agent):
    def __init__(self, pos, model, parcels):
        super().__init__(pos, model)
        #this building will be expecting how many parcels
        self.awaiting = parcels #a list
        self.val = 0

        if len(self.awaiting)!=0:
            print("Building waiting for parcels: %s" %(sorted(self.awaiting)),)
    
    def get_curb(self,pos): #returns a list of curb agents
            
            for i in self.model.grid.get_cell_list_contents([pos]):
                if type(i) is curb:
                    return i
    
    
    #pass building's parcel list to the curb border in front       
    def distribute_for_bike(self):
        curb_in_front = []
        x, y = self.pos
        if (self.pos in [(i, 27) for i in range(3,27)] ) or (self.pos in [(i, 18) for i in range(3,27)] ) or (self.pos in [(i, 9) for i in range(3,27)] ):
            curb_in_front.append((x,y+1))
            #print("Curb: i am %s" %(self.pos,))
            #print("curb in fron is %s" %curb_in_front)
            self.get_curb(curb_in_front[0]).bike_del= self.awaiting+self.get_curb(curb_in_front[0]).bike_del  
            
        elif self.pos in [(0, i) for i in range(0,30)]: #for left road
            curb_in_front.append((x+1,y))
            self.get_curb(curb_in_front[0]).bike_del= self.awaiting+self.get_curb(curb_in_front[0]).bike_del  
            
        elif self.pos in [(29, i) for i in range(1,29)]: #for right road 
            curb_in_front.append((x-1,y))
            self.get_curb(curb_in_front[0]).bike_del= self.awaiting+self.get_curb(curb_in_front[0]).bike_del  
            
    #makes all allocations once at the begining    
    def step(self):
        if self.val==0:
            self.distribute_for_bike()
            self.val=self.val+1
            
class no_d_building(Agent):
    def __init__(self, pos, model):
        super().__init__(pos, model)

# E-bike

In [7]:
class bikes(Agent):
    """ An ebike agent """
    
    def __init__(self, unique_id, model, boxes):
        super().__init__(unique_id, model)
        #state: 1= driving ,0= parked/delivering 
        self.state = 1
        self.parcels=boxes
        self.storage=[]
        self.delivered_b=[]
        self.wait_time=5
        self.spm=0
        self.tot_spm_made=0
        self.tot_spm_possible=0
        self.name = unique_id
        self.bsec_rd_p = False
        self.bt_rd_p = False
        self.delivery_time = 0

    
    def get_curb(self,pos): #returns a list of curb agents
        for i in self.model.grid.get_cell_list_contents([pos]):
            if type(i) is curb:
                return i
            
    #move agent
    def move_bike(self):
    #function for movement along road           
        x, y = self.pos

        #check which road agent is on now
        #top_road, left_road, sec_road, third_road, bot_road
        if self.pos in top_road:
            #print( 'my position is: %s'%(self.pos,))
            if self.model.grid.is_cell_empty([x-1, y])== True: #always check if cell ahead is empty or not
                self.model.grid.move_agent(self, (x-1, y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
        
                
        elif self.pos in left_road:
            if self.model.grid.is_cell_empty([x, y-1])== True:
                self.model.grid.move_agent(self, (x, y-1))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif (self.pos in sec_road) or (self.pos in third_road) or (self.pos in bot_road):
            if self.model.grid.is_cell_empty([x+1, y])== True:
                self.model.grid.move_agent(self, (x+1, y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos in right_road:
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (x, y+1))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
                
        #turning points
        elif self.pos == (2,29):
            if self.model.grid.is_cell_empty([x, y-1])== True:
                self.model.grid.move_agent(self, (2,28))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
        
        elif (self.pos == (2,20)): 
            if self.model.grid.is_cell_empty([x, y-1]) and self.model.grid.is_cell_empty([x+1, y]) == True:
                if self.bsec_rd_p == False:
                    self.model.grid.move_agent(self, (x+1,y))
                    self.bsec_rd_p = True
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (x,y-1))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                    
        elif (self.pos == (2,11)): 
            if self.model.grid.is_cell_empty([x, y-1]) and self.model.grid.is_cell_empty([x+1, y]) == True:
                if self.bt_rd_p == False:
                    self.model.grid.move_agent(self, (x+1,y))
                    self.bt_rd_p = True
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (x,y-1))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                
        elif self.pos == (2,0): 
            if self.model.grid.is_cell_empty([x+1, y])== True:
                self.model.grid.move_agent(self, (3,0))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,11): 
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (27,12))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,20): 
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (27,21))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,29): 
            if self.model.grid.is_cell_empty([x-1, y])== True:
                self.model.grid.move_agent(self, (x-1,y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,0): 
                if len(self.parcels) != 0: 
                    #later add delivery status
                    self.model.grid.move_agent(self, (27,1))
                    self.bsec_rd_p = False
                    self.bt_rd_p = False
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (28,0))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                    
        elif self.pos == (28,0): 
            self.model.grid.move_agent(self, (29,0))
            self.spm=1
            self.tot_spm_made+=1
            self.tot_spm_possible+=1
            
        else:
            print('I am stuck on the road, my pos is %s' %(self.pos,))
                
#check with the curb border, 1st parcel's loc matches with curb's 1st item
    def check_arrival(self):  
        x, y = self.pos
        if (self.pos in [(i, 29) for i in range(3,27)] ) or (self.pos in [(i, 20) for i in range(3,27)] ) or (self.pos in [(i, 11) for i in range(3,27)]): 
            location=self.get_curb((x,y-1))
            for i in self.parcels:
                for j in location.bike_del:
                    if i==j:
                        self.timer=0
                        park(self)
                        
        elif self.pos in left_road:
            location=self.get_curb((x-1,y))
            for i in self.parcels:
                for j in location.bike_del:
                    if i==j:
                        self.timer=0
                        park(self)
        elif self.pos in [(27, i) for i in range(1,29)]:
            location=self.get_curb((x+1,y))
            for i in self.parcels:
                for j in location.bike_del:
                    if i==j:
                        self.timer=0
                        park(self)
                    
#function to perform delivery               
    def deliver(self):
        #this is to keep track of time taken to deliver
        self.timer=self.timer+1
        
        #removes "delivered" parcel from list and adds it a completed delivery list for data tracking purposes
        for i in self.get_curb(self.pos).bike_del :
            for j in  self.parcels:
                if i==j:
                    self.parcels.remove(i)
                    self.delivered_b.append(i)
                    self.get_curb(self.pos).delivered_by_b.append(i)
                    
        print('Bike agent: parcel list %s' %(self.parcels,))
                    
                    

        
    #END
 
    def step(self):
        self.delivery_time += 1
        if len(self.delivered_b)!=0:
            print("Step update: Parcels delivered by bikes %s" %(self.delivered_b,))
        # moves and checks if it needs to park while driving
        if self.state==1:
            self.move_bike()
            self.check_arrival()
            
        else:
            self.deliver()
            print("Bike agent #%s" %(self.name), "to deliver: %s" %(self.parcels),)
            # waits x ticks before departing represent the amount of time it would take to complete a delivery
            if self.timer>self.wait_time:
                depart(self)
                
        self.get_curb((1,1)).dtime = [(self.name, self.delivery_time)] + self.get_curb((1,1)).dtime

# Vans

In [8]:
class vans(Agent):
    """ A van agent """
    
    def __init__(self, unique_id, model, boxes):
        super().__init__(unique_id, model)
        #state: 1= driving ,0= parked/delivering 
        self.state = 1
        self.parcels=boxes
        self.storage=[]
        self.delivered_v=[]
        self.wait_time=10
        self.stops=0
        self.spm=0
        self.tot_spm_made=0
        self.tot_spm_possible=0
        self.name = unique_id
        self.delivery_time=0
        
        self.vsec_rd_p = False
        self.vt_rd_p = False
        
    def get_curb(self,pos): #returns a list of curb agents
        for i in self.model.grid.get_cell_list_contents([pos]):
            if type(i) is curb:
                return i
            
 
    def move(self):
    #function for movement along road           
        x, y = self.pos

        #check which road agent is on now
        #top_road, left_road, sec_road, third_road, bot_road
        if self.pos in top_road:
            #print( 'my position is: %s'%(self.pos,))
            if self.model.grid.is_cell_empty([x-1, y])== True: #always check if cell ahead is empty or not
                self.model.grid.move_agent(self, (x-1, y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
        
                
        elif self.pos in left_road:
            if self.model.grid.is_cell_empty([x, y-1])== True:
                self.model.grid.move_agent(self, (x, y-1))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif (self.pos in sec_road) or (self.pos in third_road) or (self.pos in bot_road):
            if self.model.grid.is_cell_empty([x+1, y])== True:
                self.model.grid.move_agent(self, (x+1, y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos in right_road:
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (x, y+1))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
                
        #turning points
        elif self.pos == (2,29):
            if self.model.grid.is_cell_empty([x, y-1])== True:
                self.model.grid.move_agent(self, (2,28))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
        
        elif (self.pos == (2,20)): 
            if self.model.grid.is_cell_empty([x, y-1]) and self.model.grid.is_cell_empty([x+1, y]) == True:
                if self.vsec_rd_p == False:
                    self.model.grid.move_agent(self, (x+1,y))
                    self.vsec_rd_p = True
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (x,y-1))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                    
        elif (self.pos == (2,11)): 
            if self.model.grid.is_cell_empty([x, y-1]) and self.model.grid.is_cell_empty([x+1, y]) == True:
                if self.vt_rd_p == False:
                    self.model.grid.move_agent(self, (x+1,y))
                    self.vt_rd_p = True
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (x,y-1))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                
        elif self.pos == (2,0): 
            if self.model.grid.is_cell_empty([x+1, y])== True:
                self.model.grid.move_agent(self, (3,0))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,11): 
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (27,12))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,20): 
            if self.model.grid.is_cell_empty([x, y+1])== True:
                self.model.grid.move_agent(self, (27,21))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,29): 
            if self.model.grid.is_cell_empty([x-1, y])== True:
                self.model.grid.move_agent(self, (x-1,y))
                self.spm=1
                self.tot_spm_made+=1
                self.tot_spm_possible+=1
                
        elif self.pos == (27,0): 
                if len(self.parcels) != 0: 
                    #later add delivery status
                    self.model.grid.move_agent(self, (27,1))
                    self.vsec_rd_p = False
                    self.vt_rd_p = False
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                else:
                    self.model.grid.move_agent(self, (28,0))
                    self.spm=1
                    self.tot_spm_made+=1
                    self.tot_spm_possible+=1
                    
        elif self.pos == (28,0): 
            self.model.grid.move_agent(self, (29,0))
            self.spm=1
            self.tot_spm_made+=1
            self.tot_spm_possible+=1
            
        else:
            print('I am stuck on the road, my pos is %s' %(self.pos,))
            
#check with the curb border, 1st parcel's loc matches with curb's 1st item
    def check_arrival(self):  
        x, y = self.pos
        if (self.pos in [(i, 29) for i in range(3,27)] ) or (self.pos in [(i, 20) for i in range(3,27)] ) or (self.pos in [(i, 11) for i in range(3,27)]): 
            location=self.get_curb((x,y-1))
            for i in self.parcels:
                for j in location.bike_del:
                    if i==j:
                        self.timer=0
                        park(self)
        elif self.pos in left_road:
            location=self.get_curb((x-1,y))
            for i in self.parcels:
                for j in location.bike_del:
                    if i==j:
                        self.timer=0
                        park(self)
        elif self.pos in [(27, i) for i in range(1,29)]:
            location=self.get_curb((x+1,y))
            for i in self.parcels:
                for j in location.bike_del:
                    if i==j:
                        self.timer=0
                        park(self)
        
                    
#function to perform delivery               
    def deliver(self):
        #this is to keep track of time taken to deliver
        self.timer=self.timer+1
        #removes "delivered" parcel from list and adds it a completed delivery list for data tracking purposes
        for i in self.get_curb(self.pos).bike_del :
            for j in  self.parcels:
                if i==j:
                    self.parcels.remove(i)
                    self.delivered_v.append(i)
                    self.get_curb(self.pos).delivered_by_v.append(i)
                    self.wait_time+=10 #gotta change later
                    
        print('Van agent: parcel list %s' %(self.parcels,))
                    
                    
#function to depart after delivery                    
    def depart(self):
        x, y =self.pos
        #for pos in top, second, third road
        if (self.pos in [(i,28) for i in range(3,27)] ) or (self.pos in [(i,19) for i in range(3,27)] ) or (self.pos in [(i,10) for i in range(3,27)] ):
            self.model.grid.move_agent(self, (x, y+1)) #move up 1 step to move
            self.state=1
                
        elif self.pos in [(1,i) for i in range(0,30)]: #left road
            self.model.grid.move_agent(self, (x+1, y))
            self.state=1
        
        elif self.pos in [(28,i) for i in range(1,29)]: #right road
            self.model.grid.move_agent(self, (x-1, y))
            self.state=1
    
    def step(self):
        #print(self.pos)
        self.delivery_time +=1
        if len(self.delivered_v)!=0:
            print("Step update: Parcels delivered by vans %s" %(self.delivered_v,))
     # moves and checks if it needs to park while driving
        if self.state==1: #driving
            
            self.move()
            self.check_arrival()
        # waits x ticks before departing represent the amount of time it would take to complete a delivery   
        elif self.state==0: #parked and delivering
            self.deliver()
            #print("my stoppage is ",self.stoppage)
            self.timer=self.timer+1
            if self.timer>self.wait_time:
                depart(self)
        
        self.get_curb((1,1)).dtime_v = [(self.name, self.delivery_time)] + self.get_curb((1,1)).dtime_v

# Cars

In [9]:
class cars(Agent):

    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        #state: 1= drving ,0= parked
        self.state = 1
        self.timer=0
        self.wait_time=random.randint(15, 50)
        self.parcels=["JFT"]
        self.laps=4
        self.spm=0
        self.tot_spm_made=0
        self.tot_spm_possible=0

    def check_arrival(self):  
        x, y = self.pos
        park_chance=random.randint(0,100)
        
        if (self.pos in [(i, 29) for i in range(3,27)] ) or (self.pos in [(i, 20) for i in range(3,27)] ) or (self.pos in [(i, 11) for i in range(3,27)]): 
            location=get_curb(self,(x,y-1))
            if  len(self.model.grid.get_cell_list_contents([location.pos]))==1 and park_chance<=10:
                self.timer=0
                park(self)
            
        elif self.pos in left_road:
            location=get_curb(self,(x-1,y))
            if  len(self.model.grid.get_cell_list_contents([location.pos]))==1 and park_chance<=10:
                self.timer=0
                park(self)
            
        elif self.pos in [(27, i) for i in range(1,29)]:
            location=get_curb(self,(x+1,y))
            if  len(self.model.grid.get_cell_list_contents([location.pos]))==1 and park_chance<=10:
                self.timer=0
                park(self)  
            
    def step(self):
        #print(self.pos)
     # moves and checks if it needs to park while driving
        if self.state==1:
            move(self)
            self.check_arrival()
        # waits x ticks before departing represent the amount of time it would stay parked   
        elif self.state==0:
            self.timer=self.timer+1
            if self.timer>self.wait_time:
                depart(self)
        else:
            self.stoppage+=1
            park(self)

# Data collector

In [10]:
#data collection 

def comp_bike_parcels_del(model):
    parcels_del =[]
    for agent in model.schedule.agents:
        if type(agent) is curb:
            parcels_del.extend(agent.delivered_by_b)
    x = len(parcels_del)
    
    return x

def comp_van_parcels_del(model):
    parcels_del =[]
    for agent in model.schedule.agents:
        if type(agent) is curb:
            parcels_del.extend(agent.delivered_by_v)
    x = len(parcels_del)
    
    return x


def average_speed_cal(model):
    mvt=0
    possible_mvt=0.01
    for agent in model.schedule.agents:
        if type(agent) is bikes:
            possible_mvt+=1
            mvt=agent.spm+mvt
    return (mvt/possible_mvt)*21.5 #road is 6m wide, cars move at 3m/s

def average_speed_cal_total(model):
    mvt=0
    possible_mvt=0.01
    for agent in model.schedule.agents:
        if type(agent) is bikes:
            possible_mvt=agent.tot_spm_possible+possible_mvt
            mvt=agent.tot_spm_made+mvt
    return (mvt/possible_mvt)*21.5


def dtime_b(model): #return sum of agent delivery times
    ts = []
    for agent in model.schedule.agents:
        if type(agent) is curb:
            ts.extend(agent.dtime)
            a1 = dict(sorted(ts)) #dict the list to get the max value of each agent {agent id: time}
            t0 = sum(a1.values()) #len(a1.keys())
    return t0 #total time of agents taken to deliver

def dtime_v(model): #return sum of agent delivery times
    ts = []
    for agent in model.schedule.agents:
        if type(agent) is curb:
            ts.extend(agent.dtime_v)
            a1 = dict(sorted(ts))
            t0 = sum(a1.values())
    return t0 #total time of agents taken to deliver

def cost_b(model):
    t0 = dtime_b(model) #each step covered 6m
    spd = average_speed_cal(model)
    p = comp_bike_parcels_del(model)    
    alg = ((t0/p*spd*0.006)*(0.396 + (40.96/spd))) + (40.96*t0/3600) if (spd !=0 and p!=0) else 0

    return alg

def cost_v(model):
    t0 = dtime_v(model) #each step covered 6m
    spd = average_speed_cal(model)
    p = comp_bike_parcels_del(model)   
    #whole circuit is 321 blocks
    alg = ((t0/p*spd*0.006)*(11.67 + (46.31/spd))) + (46.31*t0/3600) if (spd !=0 and p!=0) else 0 #distance ran in km

    return alg

# Model

In [14]:
class ebikeModel(Model):
    """A model with some number of agents."""
    def __init__(self, height=30, width=30, N=150,BC=7,VC=35,R=1,T=3):
        
        #self.num_agents = N
        self.Total_Parcels=N
        self.Bike_Capacity=BC
        self.Van_Capacity=VC
        self.Ratio_of_vans_to_bikes=R
        self.Traffic_Level_multiplier=T

        #self.demand_for_building=D #a 1-10 scale
        
        self.Total_Bikes=find_bikes_req(self.Total_Parcels,self.Ratio_of_vans_to_bikes,self.Van_Capacity,self.Bike_Capacity)
        self.Total_Vans=find_vans_req(self.Total_Parcels,self.Ratio_of_vans_to_bikes,self.Van_Capacity,self.Bike_Capacity)
        self.Total_Cars=round(self.Traffic_Level_multiplier*20)
        self.grid = MultiGrid(width, height, False)
        self.all_vehicles=[]
        self.schedule = RandomActivation(self)
        
        self.Parcel_list=[i for i in range (1, self.Total_Parcels+1)] #for buildings
        self.Parcel_list2=[i for i in range (1, self.Total_Parcels+1)] #for ebikes
        self.Parcel_list3=[i for i in range (1, self.Total_Parcels+1)]
        
        random.shuffle(self.Parcel_list)
        random.shuffle(self.Parcel_list2)
        print("step: Parcel list will be: %s" %(self.Parcel_list2),) 
        
        
        self.removed_agents=[]
        
        road_map = np.genfromtxt("map_tranposed_V1.csv", delimiter=',')
        

        #draw map 1= road, 2=curb, 3=building, 4=non_building
        #blank space
        for _, x, y in self.grid.coord_iter():
            if road_map[x,y] == 5:
                new_road = road((x,y), self)
                self.grid.place_agent(new_road,(x, y))
                self.schedule.add(new_road)
        
        #curb
        for _, x, y in self.grid.coord_iter():
            if road_map[x,y] == 2:
                new_spot = curb((x,y), self)
                self.grid.place_agent(new_spot,(x, y))
                self.schedule.add(new_spot)
                
        #target-building
        #building_capacity = [random.randrange(11,22) for _ in range(0,56)] #all 56 buildlings have a capacity
        #random.shuffle(building_capacity) #just in case
        iter_b = 0
        for _, x, y in self.grid.coord_iter():
            if road_map[x,y] == 3:
                boxes=[]
                if random.randint(0,100) < 36:
                    for i in range(11):
                        if len(self.Parcel_list)>0:
                            boxes.append(self.Parcel_list[-1])
                            self.Parcel_list.pop()
                
                
                new_building=target_building((x,y),self,boxes)                
                
                self.grid.place_agent(new_building,(x, y))
                self.schedule.add(new_building)
                
                
                        
                
        #non-delivery-building
        for _, x, y in self.grid.coord_iter():
            if road_map[x,y] == 4:
                new_building = no_d_building((x,y), self)
                self.grid.place_agent(new_building,(x, y))
                self.schedule.add(new_building)
                
                
        #create bikes 
        for i in range(self.Total_Bikes):
            nit=[]
            for l in range(self.Bike_Capacity):
                if len(self.Parcel_list2)>0:
                
                    nit.append(self.Parcel_list2[-1])
                    self.Parcel_list2.pop()
            print ("bike's list: %s" %nit)
            a=bikes(i, self, nit)
            self.all_vehicles.append(a)
        
        print("Total bike agents: %s" %(self.Total_Bikes),)
        print("total parcels(n): %s" %(self.Total_Parcels),)  
        
        #create vans 
        for i in range(1000, 1000+self.Total_Vans):
            nit=[]
            for l in range(self.Van_Capacity):
                if len(self.Parcel_list2)>0: #yeah share same list with bikes
                
                    nit.append(self.Parcel_list2[-1])
                    self.Parcel_list2.pop()
            print ("vans list: %s" %nit)
            b=vans(i, self, nit)
            self.all_vehicles.append(b)
        
        print("Total van agents: %s" %(self.Total_Bikes),) 
        
        
        #create cars
        for i in range(3000, 3000+self.Total_Cars):
            b=cars(i, self)
            self.all_vehicles.append(b)
        
        
        #data collector
        self.datacollector = DataCollector(
        model_reporters={"Parcels Delivered by Bikes": comp_bike_parcels_del,
                         "Parcels Delivered by Vans": comp_van_parcels_del, 
                         "Average Movement Speed in Km/h":average_speed_cal,
                         "Model-time Average Speed in Km/h":average_speed_cal_total,
                         "dtime_b": dtime_b,
                         "dtime_v": dtime_v,
                         "cost_b":cost_b,
                         "cost_v": cost_v}
            )
        
           
                   
        self.running = True
        self.datacollector.collect(self)
        random.shuffle(self.all_vehicles)
        
    def step(self):
        '''Advance the model by one step.'''
       
        self.schedule.step()
        
        
        if len(self.all_vehicles)>0 and len(self.grid.get_cell_list_contents([(29,29)]))==0:
            self.grid.place_agent(self.all_vehicles[-1],(29,29))
            self.schedule.add(self.all_vehicles[-1])
            self.all_vehicles.pop()
        
        for i in self.schedule.agents:
        #last road cell
            if i.pos == (29,0):
                self.grid._remove_agent(i.pos, i)
                self.schedule.remove(i)
                self.removed_agents.append(i)
        
        self.datacollector.collect(self)
        
        if len(self.removed_agents)== self.Total_Bikes+self.Total_Vans:
            self.running = False

In [15]:
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {}
    
    if type(agent) is bikes :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.35, "Color" : "#3f37c9", "Layer":1,} #dark blue
        
    elif type(agent) is curb :
        portrayal = {"Shape": "rect",
                 "Filled": "true","Color" : "#ccd5ae",
                    "Layer": 0 ,"w":1, "h":1}
        
    elif type(agent) is vans:
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.35, "Color" : "#9315d6", "Layer":1,} #purple
    
    elif type(agent) is cars :
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5, "Color" : "#f5b342", "Layer":1,} #orange
    
    elif type(agent) is target_building :
        portrayal = {"Shape": "rect",
                 "Filled": "true", "Color": "#ee9b00",
                     "Layer": 0 ,"w":1 , "h":1}

    elif type(agent) is no_d_building:
        
        portrayal = {"Shape": "rect",
                 "Filled": "true", 
                     "Color": "#0081a7", "Layer": 0 , #turquoise
                     "w":1 , "h":1,}
    
    elif type(agent) is road:
        
        portrayal = {"Shape": "rect",
                 "Filled": "true", 
                     "Color": "#faedcd", "Layer": 0 , #cream
                     "w":1 , "h":1,}
        
    
    return portrayal
        
        
grid = CanvasGrid(agent_portrayal, 30, 30, 690,690)
#canvas grid (agent, grid_x, grid_y, pixels, pixels)
model_params = {
    "height": 30,
    "width": 30,
    "N": UserSettableParameter("slider", "Total Parcels", 150, 10, 450, 10), #initial_value, start, end, increment
    "R": UserSettableParameter("slider", "Ratio of vans to bikes", 1, 0, 5, 0.1),
    "VC": UserSettableParameter("slider", "Van Capacity", 35, 10, 60, 5),
    "BC": UserSettableParameter("slider", "Bike Capacity", 7, 1, 15 ,1),
    "T": UserSettableParameter("slider", "Traffic Level Multiplier", 2, 1, 10,1)
    
}

In [16]:
port_num = 2100

In [ ]:
port_num +=1

chart = ChartModule([{"Label": "Parcels Delivered by Bikes",
                      "Color": "#4cc279"},{"Label": "Parcels Delivered by Vans",
                      "Color": "grey"}],
                    data_collector_name='datacollector')

chart2 = ChartModule([{"Label": "Average Movement Speed in Km/h", "Color": "red"},
                      {"Label": "Model-time Average Speed in Km/h", "Color": "black"}],
                    data_collector_name='datacollector')

chart3 = ChartModule([{"Label": "dtime_b", "Color": "red"},
                      {"Label": "dtime_v", "Color": "black"}],
                    data_collector_name='datacollector')

server = ModularServer(ebikeModel,
                       [grid,chart,chart2,chart3],
                       "E Bike Model",
                        model_params)

server.port = port_num # The default
server.launch()